**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 5.4MB/s 
     |████████████████████████████████| 51kB 3.6MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [28]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example5.rdf", format="xml")

<Graph identifier=N440e892dffa44ba98a7e4ec065c55532 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [27]:
namespace = Namespace("http://somewhere#")
print("RDFLib:")

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

#######

print("SPARQL: ") 
from rdflib.plugins.sparql import prepareQuery

qu1 = prepareQuery('''
  SELECT  ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  } ''', initNs = { "rdfs": RDFS, "ns": namespace}
)

for i in g.query(qu1):
  print(i)

RDFLib:
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
SPARQL: 
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [22]:
print("RDFLib:")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
  
for s,p,o in g.triples((None, RDFS.subClassOf, namespace.Person)):
  for s2,p2,o2 in g.triples((None, RDF.type, s)):
    print(s2)

print("SPARQL:")
from rdflib.plugins.sparql import prepareQuery

qu2 = prepareQuery('''
  SELECT ?x
  WHERE { 
    { 
      ?x rdf:type ns:Person. 
    } UNION {
      ?y rdfs:subClassOf ns:Person.
      ?x rdf:type ?y
    }
  }''', initNs = { "rdfs": RDFS,"rdf": RDF, "ns": namespace}
)

for j in g.query(qu2):
  print(j)

RDFLib:
http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith
SPARQL:
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [26]:
print("RDFLib:")
for s,p,o in g.triples((None, RDF.type, namespace.Person)):
  for s2,p2,o2 in g.triples((s, None, None)):
      print(s2,p2,o2)

for s,p,o in g.triples((None, RDFS.subClassOf, namespace.Person)):
  for s2,p2,o2 in g.triples((None, RDF.type, s)):
    for s3,p3,o3 in g.triples((s2, None, None)):
      print(s3,p3,o3)

print("SPARQL:")
from rdflib.plugins.sparql import prepareQuery

qu3 = prepareQuery('''
  SELECT ?s ?o ?p
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?o ?p
    } UNION {
      ?s rdf:type ?s2.
      ?s ?o ?p.
      ?s2 rdfs:subClassOf ns:Person
    }
  } ''', initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for k in g.query(qu3):
  print(k)

RDFLib:
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewhere#h